# Why NVIS antenna configuration?
## Properties
* easy to setup : no installation on height needed
* hard to jam and locate (direction finding) : energy goes straight up
* simple and cheap antenna : horizontal dipole
* omnidirectional coverage
* works well with low power
* range limited to a few 100 kms
* frequency limited to 2-10MHz, depending on ionospheric conditions
    * 4 - 8MHz during day : 40m band
    * 2 - 4MHz at night : 80m band
* other party should also have NVIS antenna

## Frequencies
* Must be high enough, otherwise too much D-layer absorption
* Must be low enough to be refracted by F-layer.  Maximum frequency is the Ionospheric foF2 critical frequency.
* [Space Weather Services - Ionospheric Map](https://www.sws.bom.gov.au/HF_Systems/6/5)
* 80m band seems to be the only viable option in western Europe

## Setup
* Radiation angle (typically 75° for NVIS) increases as active element is lowered (typicallyλ 0.10 to 0.25λ above ground)
* Mounting horizontal dipole lower means more ground losses and need for retuning (shortening)

## Construction of a 40m-60m-80m dipole
### Materials
* Wire : 
    * #AWG20 (0.5mm²) to #AWG26 (0.13mm²) used for mobile installation
    * Stranded copper for ease of handling
    * For 40m dipole, buy length of 25m
* 2 insulators : 2 pieces of FR4, with 3 holes spaced about 15mm apart (works probably better than egg-insulator or dog-bone insulator for stranded wire)

## U01 Antenna
* [U01 emergency communications Antenna - 80/60/40/20/17/15/10m EFHW](https://dl2man.de/u01-emergency-communications-antenna/)
* Sloped EFHW
* Uses ferrite core :
    * [Amidon FT82-43](https://www.reichelt.com/be/en/ferrite-core-1-29-h-material-43-ami-ft-82-43-p245914.html): ⌀21mm, size core needed for 5W operation
    * [Fair-rite 5943001301](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5943001301/8599633): ⌀25.4mm 
    
## Antenna calibration
* [Revised - Use of The NanoVNA For Antenna Measurements W5IFQ 7 AUG 2021](https://austinhams.org/wp-content/uploads/2021/08/Use-of-the-NanoVNA-for-Antenna-Measurements-AARC.pdf)

## References
* [R&S Understanding NVIS](https://www.youtube.com/watch?v=k3IFj_Z5cLM)
* [Local Emergency Communications with NVIS - Let's Build an 80m/40m Dipole](https://www.youtube.com/watch?v=yYXVKtu3nwk)
* [Receiving WSPR on SDRplay RSPduo Shortwave Antenna Performance Test](https://www.youtube.com/watch?v=8lZds2MTKTI)